# Singal To Noise All Models
 - Calculates the singal to nosie for all models in longrunmip for GMST
 - Fraction of models stable for windows for GMST
 
 PHD-9:
 * read_and_merge_netcdfs moved from longrunmip_loader to open_ds


In [9]:
import warnings

import xarray as xr
import numpy as np
from importlib import reload
import string

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker

import os
import sys

sys.path.append('../../..')
import constants

sys.path.append(constants.MODULE_DIR)

from classes import ExperimentTypes
import utils
import xarray_extender as xce
import signal_to_noise as sn
import open_ds
import xarray_class_accessors as xca
import sn_plotting as sn_plot

warnings.filterwarnings('ignore')

logger = utils.get_notebook_logger()

IndentationError: unindent does not match any outer indentation level (sn_plotting.py, line 195)

In [ ]:
from classes import ExperimentTypes


In [ ]:
constants.EXPERIMENTS_TO_RUN

In [ ]:
notebook_number='03_00'
experiment_params = constants.EXPERIMENTS_TO_RUN[0] # 0, 1, 2 3, 4, 5, 6, 7
experiment_params

In [ ]:
ROOT_DIR = os.path.join(constants.LONGRUNMIP_DIR, experiment_params["variable"], 'regrid_retimestamped')
ROOT_DIR

In [ ]:
units = constants.VARIABLE_INFO[experiment_params["variable"]]["units"]
longname = constants.VARIABLE_INFO[experiment_params["variable"]]['longname']
print(f'{longname=}', f'{units=}', sep=' , ')

# Loading Data

In [ ]:
models_to_get = open_ds.get_models_longer_than_length()
utils.pprint_list(models_to_get, num_start_items=7)

In [ ]:
files_to_open_experiment = open_ds.get_file_names_from_from_directory(
    ROOT_DIR, ExperimentTypes.ABRUPT4X, models_to_get)

files_to_open_control = open_ds.get_file_names_from_from_directory(
    ROOT_DIR, ExperimentTypes.CONTROL, models_to_get)

utils.pprint_list(files_to_open_experiment, num_start_items=5)

In [7]:
control_ds = open_ds.read_and_merge_netcdfs(files_to_open_control, ROOT_DIR, 
                                            mask=experiment_params['mask'])
abrupt4x_ds = open_ds.read_and_merge_netcdfs(files_to_open_experiment, ROOT_DIR,
                                             mask=experiment_params['mask'])
abrupt4x_ds

NameError: name 'open_ds' is not defined

In [ ]:
models = list(abrupt4x_ds.data_vars)
num_cols = 3
num_rows = int(np.ceil(len(models)/num_cols))

fig = plt.figure(figsize=(16,10))

for i,model in enumerate(models):
    ax = fig.add_subplot(num_rows, num_cols, i+1)
    
    abrupt4x_ds[model].isel(time=0).plot(ax=ax)
    ax.set_title(model)
    
    ax.set_title(model)

In [ ]:
abrupt4x_mean,control_ds_mean = sn.calculate_global_value(abrupt4x_ds, control_ds, 
                                                          experiment_params["variable"],
                                                constants.HEMISPHERE_LAT[experiment_params['hemisphere']])

In [ ]:
constants.MULTI_WINDOW_RUN_PARAMS

In [ ]:
max_effective_length =  constants.LONGRUNMIP_LENGTH#sn.get_upper_lim(abrupt4x_mean, constants.MULTI_WINDOW_RUN_PARAMS['end_window'])
max_effective_length

In [ ]:
%%time
stable_sn_ds, unstable_sn_ds = abrupt4x_mean.sn_ds.sn_multiwindow(
    control_ds_mean, parallel=True, **constants.MULTI_WINDOW_RUN_PARAMS)

In [ ]:
number_models_present = sn.count_over_data_vars(abrupt4x_mean, list(unstable_sn_ds.data_vars), dim='model')

unstable_number_da = sn.number_finite(unstable_sn_ds)
stable_number_da = sn.number_finite(stable_sn_ds)

unstable_number_da

In [8]:
%%time
stable_point_ds = sn.get_dataset_stable_year_multi_window(unstable_sn_ds).rename({'variable': 'model'})
stable_point_ds

NameError: name 'sn' is not defined

In [9]:
# stable_point_ds = xr.where(stable_point_ds.time > max_effective_length, stable_point_ds, stable_point_ds.time)

# Plots

### Mean Temp Plots

In [10]:
hemisphere_title = string.capwords(experiment_params['hemisphere'].replace('_', ' '))
hemisphere_title

NameError: name 'string' is not defined

In [ ]:
mask_title = f'({experiment_params["mask"]} only)' if experiment_params["mask"] else ''
mask_title

In [ ]:
run_tag = f'{notebook_number}_{experiment_params["variable"]}' +\
        f'_{experiment_params["hemisphere"]}_mask_{experiment_params["mask"]}'
run_tag

In [ ]:
plt.style.use('seaborn-darkgrid')
fig, ax = plt.subplots(figsize=(15,8))

time = control_ds_mean['time'].values
years = [t.year for t in time]
for var in list(control_ds_mean.data_vars):

    plot_kwargs = dict(alpha = 0.6,linewidth = 2)
        
    ax.plot(years, control_ds_mean[var].values, label=var, **plot_kwargs) #  - 273.15
    
ax.set_ylabel(f'{hemisphere_title} Mean\n{longname} ({units})', size=18, labelpad=20)
ax.set_xlabel('Time (Years)', size=18)
ax.set_xlim(years[0], years[500])
ax.tick_params(axis='x', labelsize=14)
leg = ax.legend(ncol = 5, fontsize=16)
[line.set_linewidth(3) for line in leg.get_lines()];
ax.set_title(f'Pre-Industrial Control {hemisphere_title} Mean {longname} {mask_title}', fontsize=25);

fig.savefig(os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL, f'{run_tag}__mean_control.png'))

In [ ]:
plt.style.use('seaborn-darkgrid')

data = abrupt4x_mean.where(abrupt4x_mean.time.dt.year < 13500, drop = True)

fig, ax = plt.subplots(figsize=(15,8))

for var in list(data.data_vars):
    
    # Removing nans from data
    da = data[var].dropna(dim='time')
    time = da.time.values
    years = [t.year for t in time]

    values = da.values
    
    line_kwargs = dict(alpha = 0.6,linewidth = 2)
    
    ax.plot(years, values, label=var, **line_kwargs)
    c = plt.gca().lines[-1].get_color()
#     ax.annotate(var, xy = (time[-1], values[-1]), color=c, zorder=1000, size=12)

ax.set_ylabel(f'{hemisphere_title} Mean {longname} Anomaly ({units})', size=16)
ax.set_xlabel('Time (years)', size=16);
ax.set_title(f'{hemisphere_title} Abrupt 4xCO2 Mean {experiment_params["variable"]} {mask_title}', fontsize=15);

# ax.set_ylim([0,15]);|
ax.set_xlim(years[0], years[max_effective_length])
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)
leg = ax.legend(ncol=4, fontsize = 16);
[line.set_linewidth(3) for line in leg.get_lines()];
fig.savefig(os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL, f'{run_tag}_mean_abrupt4x.png'))

### Signal to Noise

In [ ]:
models = [model for model in list(constants.MODEL_PARAMS) if model in stable_point_ds.model.values]
models

In [ ]:
plt.style.use('seaborn-darkgrid')
fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(111)

for model in models:
    color = constants.MODEL_PARAMS[model]['color']
    ECS = constants.MODEL_PARAMS[model]['ECS']
    label = f'{model} ({ECS}K)' 
    da = stable_point_ds.sel(model=model).time.plot(ax=ax,y='window', label=label, color=color, linewidth=1.5,
                                                   alpha=0.8)
    
stable_point_ds.median(dim='model').time.plot(ax=ax,y='window', label='median', color='k', linewidth=1.5) 

ax.set_xlim(0, max_effective_length + 1)
ax.set_ylim(20, constants.MULTI_WINDOW_RUN_PARAMS['end_window'])
leg = ax.legend(ncol=1, bbox_to_anchor=[1, 0.857], frameon=True, facecolor='white', fontsize=14)
leg.set_title('Model')
leg.get_title().set_fontsize('10')
title = f'Year of Stabilistaion for {longname} {mask_title}' #{hemisphere_title} 
ax.set_title(title, fontsize=25)
ax.set_xlabel('Year of Stabilisation', fontsize=18)
ax.set_ylabel('Window Length (years)', fontsize=18)
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)


save_name = os.path.join(
    constants.IMAGE_SAVE_DIR_TOP_LEVEL, f'{run_tag}_all_models_stable_year.png')

fig.savefig(save_name, bbox_inches='tight', dpi=200)

In [ ]:
models

In [ ]:
reload(sn_plot)


In [ ]:
reload(sn_plot)
mpl.rcParams.update(mpl.rcParamsDefault)

for model in list(models):
    print(model)
    ECS = constants.MODEL_PARAMS[model.lower()]['ECS']
    title = f'{hemisphere_title} {longname.capitalize()} Signal-to-Noise for {model} {mask_title}(ECS={ECS}K)'
    (fig, ax1, ax2, ax3, cbar) = sn_plot.sn_multi_window_in_time(unstable_sn_ds[model], 
                            stable_sn_ds[model], abrupt4x_mean[model].dropna(dim='time'),
                            stable_point_ds = stable_point_ds.sel(model=model),
                            title=title, step=1, xupperlim=max_effective_length, cbar_label='Signal-to-Noise',
                            ax2_ylabel = f'{hemisphere_title} Mean\n{longname} Anomaly ({units})',
                                                                logginglevel='ERROR');

#     stable_point_ds.sel(model=model).time.plot(y='window',  ax=ax1, color='k')
#     ax1.set_title('')
#     ax1.set_ylabel('Window Length (Years)', fontsize=15)
#     ax1.set_xlabel('Time (Years)', fontsize=16)
#     ax2.set_xlabel('Time (Years)', fontsize=16)
    
    save_name = os.path.join(
        constants.IMAGE_SAVE_DIR_TOP_LEVEL, 
        f'{run_tag}_{model}_signal_to_noise_multiwindow.png')
    
    print(f'Saving {save_name}')
    fig.savefig(save_name, dpi=400, bbox_inches='tight')

In [ ]:
experiment_params["variable"] in ['sic', 'surf']

In [ ]:
cmap = "Blues" if experiment_params["variable"] in ['sic', 'surf'] else 'Reds'
cmap

In [ ]:
reload(sn_plot)
mpl.rcParams.update(mpl.rcParamsDefault)
fig, ax1, ax2, ax3,cbar = sn_plot.sn_multi_window_in_time(
    unstable_number_da, stable_number_da, abrupt4x_mean, 
    logginglevel='ERROR', 
    hieght = 15, width = 12,
    vmin=0, vmax=len(models)+1, step=1, cbartick_offset = 0.5,
    cmap=cmap,
    extend='neither', xupperlim = max_effective_length,
    cbar_label= 'Number of Unstable Models', 
    ax2_ylabel = f'Global Mean {longname} Anomaly ({units})',
    title = f'{hemisphere_title} Fraction of Models Stable for {longname} {mask_title}',
    cbar_ncols = 3, filter_max=True)

# ax1.set_ylabel('Window Length (Years)', fontsize=18)

save_name = os.path.join(
        constants.IMAGE_SAVE_DIR_TOP_LEVEL, 
        f'{run_tag}_all_models_signal_to_noise_multiwindow.png')
print(save_name)
fig.savefig(save_name , dpi=200, bbox_inches='tight')